In [1]:
import pandas as pd
import json
import os

import warnings 
warnings.filterwarnings("ignore")
from importlib import reload

# this ensures that I can update the class without losing my variables in my notebook
import xenium_cluster
reload(xenium_cluster)
from xenium_cluster import XeniumCluster
from utils.metrics import *

In [2]:
# Path to your .gz file
file_path = 'data/hBreast/transcripts.csv.gz'

# Read the gzipped CSV file into a DataFrame
df_transcripts = pd.read_csv(file_path, compression='gzip')

# drop cells without ids
df_transcripts = df_transcripts[df_transcripts["cell_id"] != -1]

# drop blanks and controls
df_transcripts = df_transcripts[~df_transcripts["feature_name"].str.startswith('BLANK_') & ~df_transcripts["feature_name"].str.startswith('NegControl')]

In [3]:
def run_experiment(data, dataset_name: str, current_spot_size: int, third_dim: bool, resolutions: list, n_clusters=15):
    
    clustering = XeniumCluster(data=data, dataset_name=dataset_name)
    clustering.set_spot_size(current_spot_size)
    clustering.create_spot_data(third_dim=third_dim, save_data=True)

    print(f"The size of the spot data is {clustering.xenium_spot_data.shape}")

    clustering.normalize_counts(clustering.xenium_spot_data)
    clustering.generate_neighborhood_graph(clustering.xenium_spot_data, plot_pcas=False)

    Louvain_cluster = clustering.Louvain(clustering.xenium_spot_data, resolutions, embedding="umap", save_plot=True)

    return clustering, Louvain_cluster

In [4]:
import numpy as np
import torch
from scipy.spatial.distance import cdist

def record_results(original_data, cluster_dict, results_dir, model_name, filename, spot_size, third_dim, K=None, resolution=None, uses_spatial=True):

    dirpath = f"{results_dir}/{model_name}{'/' + (str(resolution) if resolution is not None else str(K))}/clusters/{spot_size}"
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

    filepath = f"{dirpath}/{filename}.json"

    with open(filepath, "w") as f:
        json.dump(cluster_dict[model_name], f, indent=4)

    wss = {}
    if resolution is not None:
        current_clustering = np.array(cluster_dict[model_name][spot_size][third_dim].get(
            resolution, 
            cluster_dict[model_name][spot_size][third_dim]
        ))
    else:
        current_clustering = np.array(cluster_dict[model_name][spot_size][third_dim][uses_spatial].get(
            K, 
            cluster_dict[model_name][spot_size][third_dim][uses_spatial]
        ))
    cluster_labels = np.unique(current_clustering)

    original_data.xenium_spot_data.obs[f"{model_name} cluster"] = np.array(current_clustering)
    # Extracting row, col, and cluster values from the dataframe
    rows = torch.tensor(original_data.xenium_spot_data.obs["row"].astype(int))
    cols = torch.tensor(original_data.xenium_spot_data.obs["col"].astype(int))
    clusters = torch.tensor(original_data.xenium_spot_data.obs[f"{model_name} cluster"].astype(int))
    cluster_labels = np.unique(clusters)
    num_clusters = len(cluster_labels)

    num_rows = int(max(rows) - min(rows) + 1)
    num_cols = int(max(cols) - min(cols) + 1)

    cluster_grid = torch.zeros((num_rows, num_cols), dtype=torch.int)

    cluster_grid[rows, cols] = torch.tensor(clusters, dtype=torch.int)

    for label in cluster_labels:
        current_cluster_locations = torch.stack(torch.where((cluster_grid == label)), axis=1).to(float)
        wss[f"Cluster {label}"] = (spot_size ** 2) * torch.mean(torch.cdist(current_cluster_locations, current_cluster_locations)).item()
        print(f"POSSIBLE {len(cluster_labels)}", label, wss[f"Cluster {label}"])

    wss_dirpath = f"{results_dir}/{model_name}{'/' + (str(resolution) if resolution is not None else str(K))}/wss/{spot_size}/"
    if not os.path.exists(wss_dirpath):
        os.makedirs(wss_dirpath)

    wss_filepath = f"{wss_dirpath}/{filename}_wss.json"
    with open(wss_filepath, "w") as f:
        json.dump(wss, f, indent=4)

In [5]:
cluster_dict = {"Louvain": {}}
wss = {"Louvain": {}}
results_dir = "results/hBreast"
cluster_results_filename = "clusters_w_plots_7_26"

In [6]:
import matplotlib
matplotlib.use('Agg')

resolutions = [0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0]
for spot_size in [50, 75, 100]:
    for third_dim in [False]:
        for K in [17]:
            original_data, Louvain_cluster = run_experiment(df_transcripts, "hBreast", spot_size, third_dim, resolutions, n_clusters=K)

            # Louvain
            if "Louvain" not in cluster_dict:
                cluster_dict["Louvain"] = {}
            if spot_size not in cluster_dict["Louvain"]:
                cluster_dict["Louvain"][spot_size] = {}
            cluster_dict["Louvain"][spot_size][third_dim] = {res: clusters.tolist() for res, clusters in Louvain_cluster.items()}
            for resolution in resolutions:
                record_results(original_data, cluster_dict, results_dir, "Louvain", cluster_results_filename, spot_size, third_dim, K, resolution)

            print(f"Cluster with spot size {(spot_size, third_dim, K)} completed.")

The size of the spot data is (23444, 280)


KeyboardInterrupt: 

In [ ]:
spot_sizes = [50,75,100]
resolutions = [0.25, 0.5, 0.75, 1.0]
in_billions = 1_000_000_000
method = "Louvain"
for spot_size in spot_sizes:
    for K in [17]:
        for resolution in resolutions:
            filename = f"results/hBreast/{method}/{resolution}/wss/{spot_size}/{cluster_results_filename}_wss.json"
            if os.path.exists(filename):
                with open(filename, "r") as wss_dict:
                    current_wss = json.load(wss_dict)
                print("Method:", method, "Spot Size", spot_size, "Resolution", resolution, "Num Clusters:", len(current_wss), "Total WSS", sum(current_wss.values()) / in_billions)

HERE
Method: K-Means Spot Size 100 Num Clusters: 17 Total WSS 0.005813899414138968
HERE
Method: K-Means_No_Spatial Spot Size 100 Num Clusters: 17 Total WSS 0.28332006679553495
HERE
Method: Hierarchical Spot Size 100 Num Clusters: 17 Total WSS 0.00560841868163628
HERE
Method: Hierarchical_No_Spatial Spot Size 100 Num Clusters: 17 Total WSS 0.28219239547169006
Method: Leiden Spot Size 100 Resolution 0.25 Num Clusters: 6 Total WSS 0.09779246825275083
Method: Leiden Spot Size 100 Resolution 0.5 Num Clusters: 9 Total WSS 0.12700534134305
Method: Leiden Spot Size 100 Resolution 0.75 Num Clusters: 14 Total WSS 0.2060176697480106
Method: Leiden Spot Size 100 Resolution 1.0 Num Clusters: 17 Total WSS 0.2464717861724789
Method: Louvain Spot Size 100 Resolution 0.25 Num Clusters: 4 Total WSS 0.0661864015541885
Method: Louvain Spot Size 100 Resolution 0.5 Num Clusters: 8 Total WSS 0.12385353960127954
Method: Louvain Spot Size 100 Resolution 0.75 Num Clusters: 10 Total WSS 0.15173654314022136
Metho